## Imports and Functions

In [3]:
# imports
import pandas
import pickle
from googletrans import Translator, constants
import requests
import pandas
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import gzip
import json


translator = Translator()

from openai.embeddings_utils import (
    get_embedding,
    distances_from_embeddings,
    tsne_components_from_embeddings,
    chart_from_components,
    indices_of_nearest_neighbors_from_distances,
)
import openai
from dotenv import load_dotenv
import os
load_dotenv()

# constants
EMBEDDING_MODEL = "text-embedding-ada-002"

openai.api_key = os.environ.get('OPENAI_API_KEY')


def get_key_words(text):
    """
        Returns keywords extracted from course information
    """
    
    text = "Esta es la información de un curso online. Escribe en una línea y en inglés las 5 palabras clave\
     que describen el tema que trata el curso. Las 5 palabras deben describir el tema del curso de la manera más específica posible. "+text
    max_tokens = 4097

    if len(text)> 4097*2.5:
        text = text[:int(max_tokens*2)]
        print("TRUNCATED: ", len(text))
    
    try:
        response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": text}
                ]
            )

        key_words = response["choices"][0]["message"]["content"]
    except Exception as e:
        print("ERROR: ", e)
        key_words = ""
    return key_words


def embedding_from_string(
    string: str,
    model: str = EMBEDDING_MODEL,
) -> list:
    """
        Returns embedding from string
    """
    embedding = get_embedding(string, model)
    
    return embedding
    
def get_element_content(element):
    return str(element["content"]) 
    



## Load elements from pickle

In [ ]:
source_file = "source.pickle"
source_elements = []
try:
    with open(source_file, 'rb') as handle:
        source_elements = pickle.load(handle)
except:
    print("No source file found")

## Get Keywords from source elements

In [9]:
keywords_data = {'id': [], 'name': [], 'keywords': []}

keywords_file = "keywords.pickle"

try:
    with open(keywords_file, 'rb') as handle:
        key_words = pickle.load(handle)
        
except:
    print("No current keywords found")
    
print("TOTAL ELEMENTS: ", len(source_elements))
current_keywords = len(key_words['Element'])
counter = current_keywords
for element in source_elements[current_keywords:]:
    print("element: "+str(counter)+" of "+str(len(source_elements)))
    key_words = get_key_words(element)
    
    if key_words!="":
        keywords_data['id'].append(element['id'])
        keywords_data['name'].append(element['name'])
        with open(keywords_file, 'wb') as handle:
            pickle.dump(keywords_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
    else:
        break
        
    counter = counter + 1
    

TOTAL COURSES:  258
curso: 192 de 258
curso: 193 de 258
curso: 194 de 258
curso: 195 de 258
curso: 196 de 258
curso: 197 de 258
curso: 198 de 258
curso: 199 de 258
curso: 200 de 258
curso: 201 de 258
curso: 202 de 258
curso: 203 de 258
curso: 204 de 258
curso: 205 de 258
curso: 206 de 258
curso: 207 de 258
curso: 208 de 258
curso: 209 de 258
curso: 210 de 258
curso: 211 de 258
curso: 212 de 258
curso: 213 de 258
curso: 214 de 258
curso: 215 de 258
curso: 216 de 258
curso: 217 de 258
curso: 218 de 258
curso: 219 de 258
curso: 220 de 258
curso: 221 de 258
curso: 222 de 258
curso: 223 de 258
curso: 224 de 258
curso: 225 de 258
curso: 226 de 258
curso: 227 de 258
curso: 228 de 258
curso: 229 de 258
curso: 230 de 258
curso: 231 de 258
curso: 232 de 258
curso: 233 de 258
curso: 234 de 258
curso: 235 de 258
curso: 236 de 258
curso: 237 de 258
curso: 238 de 258
curso: 239 de 258
curso: 240 de 258
curso: 241 de 258
curso: 242 de 258
curso: 243 de 258
curso: 244 de 258
curso: 245 de 258
curso: 2

## Write Keywords in excel

In [10]:
## Writting the KEY WORDS in excel:
keywords_excel_file = "keywords_excel.xlsx"
    
with open(keywords_file, 'rb') as handle:
    keywords_data = pickle.load(handle)
    
    
df_en = pandas.DataFrame(keywords_data)

 
writer = pandas.ExcelWriter(keywords_excel_file, engine='xlsxwriter')
df_en.to_excel(writer, sheet_name = "EN")

writer.close()

## Generate element embeddings from their Keywords

In [11]:
embeddings_file = "embeddings_data.gz"

with open(keywords_file, 'rb') as handle:
    keywords_data= pickle.load(handle)
    
## Convert from dataframe to list of objects:
keywords_data = []
for i in range(len(keywords_data['name'])):
    name = keywords_data['name'][i]
    element_id = keywords_data['id'][i]
    element_keywords = keywords_data['keywords'][i]
    keywords_data.append({'id':element_id, 'name':name, 'keywords': element_keywords})
    
counter = 1
index = 0
for element in keywords_data:
    print("Element "+str(counter)+" of "+str(len(keywords_data)))
    embeddings = embedding_from_string(element['keywords'])
    keywords_data[index]["embeddings"] = embeddings
    
    with gzip.open(embeddings_file, 'wb') as f:
        # Write keywords data in zip file
        pickle.dump(keywords_data, f)
        
    counter = counter + 1
    index = index + 1
    



Course 1 of 258
Course 2 of 258
Course 3 of 258
Course 4 of 258
Course 5 of 258
Course 6 of 258
Course 7 of 258
Course 8 of 258
Course 9 of 258
Course 10 of 258
Course 11 of 258
Course 12 of 258
Course 13 of 258
Course 14 of 258
Course 15 of 258
Course 16 of 258
Course 17 of 258
Course 18 of 258
Course 19 of 258
Course 20 of 258
Course 21 of 258
Course 22 of 258
Course 23 of 258
Course 24 of 258
Course 25 of 258
Course 26 of 258
Course 27 of 258
Course 28 of 258
Course 29 of 258
Course 30 of 258
Course 31 of 258
Course 32 of 258
Course 33 of 258
Course 34 of 258
Course 35 of 258
Course 36 of 258
Course 37 of 258
Course 38 of 258
Course 39 of 258
Course 40 of 258
Course 41 of 258
Course 42 of 258
Course 43 of 258
Course 44 of 258
Course 45 of 258
Course 46 of 258
Course 47 of 258
Course 48 of 258
Course 49 of 258
Course 50 of 258
Course 51 of 258
Course 52 of 258
Course 53 of 258
Course 54 of 258
Course 55 of 258
Course 56 of 258
Course 57 of 258
Course 58 of 258
Course 59 of 258
Course